#Benchmark of Spatiotemporal Graph Neural Networks for Short-Term Load Forecasting


#Install basic dependencies.

In [ ]:
# Here's a single command to uninstall *all* of the packages you installed for STGNN and its dependencies:
!pip uninstall -y \
    numpy pandas scipy matplotlib plotnine mizani \
    torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric \
    pytorch-lightning torch-spatiotemporal \
    gluonts transformers sktime jax jaxlib huggingface-hub hydra-core jaxtyping multiprocess python-dotenv einops dtw-python websockets fsspec gcsfs pyinform optuna h5py typing-extensions wheel
!pip install numpy==1.26.4 pandas==2.2.2
!pip install --upgrade pandas pandasai

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: matplotlib 3.10.0
Uninstalling matplotlib-3.10.0:
  Successfully uninstalled matplotlib-3.10.0
Found existing installation: plotnine 0.14.5
Uninstalling plotnine-0.14.5:
  Successfully uninstalled plotnine-0.14.5
Found existing installation: mizani 0.13.5
Uninstalling mizani-0.13.5:
  Successfully uninstalled mizani-0.13.5
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
U

#Unzip tower data zip file.

In [ ]:
!unzip 'IoBT All Tower Data.zip'

Archive:  IoBT All Tower Data.zip
   creating: IoBT All Tower Data/
  inflating: IoBT All Tower Data/MSA Tower LAT LONG.CSV  
  inflating: IoBT All Tower Data/tower10_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower11_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower12_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower13_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower14_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower15_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower18_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower1_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower21_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower22_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower23_data_processed_1to9days.csv  
  inflating: IoBT All Tower Data/tower24_data_processed_1to9days.csv  
  inflating: IoBT All T

In [ ]:
!ls 'IoBT All Tower Data' | wc -l

30


In [ ]:
import pandas as pd
import os

all_towers = []
root = 'IoBT All Tower Data'
features = ["HourlyDateTime", "Temperature_2m", "BarometricPressure", "RelativeHumidity_2m", "Rainfall"]

def group_data_hourly(data):
    # Clean and format the 'Date' column (remove trailing ".0")
    data['Date'] = data['Date'].astype(str).str.replace(r'\.0$', '', regex=True)

    # Clean and format 'Time' to remove any fractional part and ensure it's zero-padded correctly
    data['Time'] = data['Time'].apply(lambda x: '{:04}'.format(int(float(x))))

    # Combine 'Date' and formatted 'Time' columns correctly to create a datetime column
    data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%Y%m%d %H%M', errors='coerce')

    # Remove rows that could not be converted
    if data['DateTime'].isnull().any():
        num_invalid = data['DateTime'].isnull().sum()
        print(f"Warning: {num_invalid} rows had invalid DateTime and will be dropped.")
        data = data.dropna(subset=['DateTime'])

    # Round datetime to the nearest hour
    data['HourlyDateTime'] = data['DateTime'].dt.round('H')

    # Group by hourly datetime and aggregate using mean for specified columns
    hourly_aggregated_data = data.groupby('HourlyDateTime').agg({
        'BatteryVoltage': 'mean',
        'PanelTemperature': 'mean',
        'BarometricPressure': 'mean',
        'Rainfall': 'mean',
        'Temperature_2m': 'mean',
        'RelativeHumidity_2m': 'mean',
        'Temperature_10m': 'mean',
        'SolarRadiation_W/m2': 'mean',
        'SolarRadiation_kW/m2': 'mean'
    }).reset_index()

    return hourly_aggregated_data
towerMap = dict()
allFiles = []
# Process each CSV file
for file in os.listdir(root):
    if file.endswith("1to9days.csv"):
        allFiles.append(file)
        towerName = file.split("_")[0]
        if towerName not in towerMap:
          towerMap[towerName] = []
        file_path = os.path.join(root, file)
        tower = pd.read_csv(file_path)
        tower_aggregated = group_data_hourly(tower)
        towerMap[towerName] = tower_aggregated[features]
        all_towers.append(tower_aggregated[features])

# Optionally, combine all towers into one DataFrame
combined_towers_df = pd.concat(all_towers, ignore_index=True)
print(towerMap.keys())


<ipython-input-3-0114900fe0a0>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HourlyDateTime'] = data['DateTime'].dt.round('H')


dict_keys(['tower13', 'tower29', 'tower4', 'tower18', 'tower24', 'tower31', 'tower2', 'tower22', 'tower12', 'tower25', 'tower14', 'tower10', 'tower30', 'tower6', 'tower8', 'tower32', 'tower5', 'tower28', 'tower7', 'tower9', 'tower26', 'tower15', 'tower11', 'tower36', 'tower21', 'tower3', 'tower23', 'tower1', 'tower27'])


In [9]:
# Iterate over a copy of the dictionary items
for name, val in list(towerMap.items()):
    # If either dimension does not meet the expected value, delete it
    if val.shape[0] != 217 or val.shape[1] != 5:
      for files in allFiles:
        if name+"_" in files:
            del towerMap[name]
            del allFiles[allFiles.index(files)]
            print(f"Deleting tower {name} with shape {val.shape} and path {files}")

# Get the number of towers remaining in towerMap
print(towerMap.keys())
print(len(allFiles))

dict_keys(['tower13', 'tower4', 'tower18', 'tower24', 'tower31', 'tower2', 'tower12', 'tower25', 'tower14', 'tower10', 'tower30', 'tower6', 'tower8', 'tower32', 'tower5', 'tower28', 'tower7', 'tower9', 'tower26', 'tower15', 'tower11', 'tower21', 'tower3', 'tower23', 'tower27'])
25


#Function to group data by a sampling rate 'X' (in mins)

In [10]:
def group_data_by_interval(data, round_interval, datetime_col_name):
    features = [datetime_col_name, "Temperature_2m", "BarometricPressure", "RelativeHumidity_2m", "Rainfall"]

    """
    Aggregate sensor data based on a specified time rounding interval.

    Parameters:
      data (pd.DataFrame): Source data including 'Date', 'Time', optionally 'Timestamp'.
      round_interval (str): Rounding frequency (e.g., '15min', '45min', '5min').
      datetime_col_name (str): Name for the new rounded datetime column.

    Returns:
      pd.DataFrame: Aggregated data grouped by rounded datetime.
    """

    # Clean 'Date' (remove trailing ".0")
    data['Date'] = data['Date'].astype(str).str.replace(r'\.0$', '', regex=True)

    # Ensure 'Time' has no fractional part and is correctly zero-padded
    data['Time'] = data['Time'].apply(lambda x: '{:04}'.format(int(float(x))))

    # Combine 'Date' and cleaned 'Time' into datetime
    data['DateTime'] = pd.to_datetime(
        data['Date'] + ' ' + data['Time'],
        format='%Y%m%d %H%M',
        errors='coerce'
    )

    # Drop rows with invalid datetime
    if data['DateTime'].isnull().any():
        num_invalid = data['DateTime'].isnull().sum()
        print(f"Warning: {num_invalid} rows had invalid DateTime and will be dropped.")
        data = data.dropna(subset=['DateTime'])

    # Add seconds after hour from 'Timestamp' column if present
    if 'Timestamp' in data.columns:
        data['SecondsAfterHour'] = (data['Timestamp'].astype(float) % 3600)
        data['DateTime'] += pd.to_timedelta(data['SecondsAfterHour'], unit='s')

    # Round datetime to specified interval and create new datetime column
    data[datetime_col_name] = data['DateTime'].dt.round(round_interval)

    # Group by rounded datetime and aggregate measurements
    aggregated_data = data.groupby(datetime_col_name).agg({
        'BatteryVoltage': 'mean',
        'PanelTemperature': 'mean',
        'BarometricPressure': 'mean',
        'Rainfall': 'mean',
        'Temperature_2m': 'mean',
        'RelativeHumidity_2m': 'mean',
        'Temperature_10m': 'mean',
        'SolarRadiation_W/m2': 'mean',
        'SolarRadiation_kW/m2': 'mean'
    }).reset_index()

    return aggregated_data[features]


#Create a dictionary to map sampling intervals to data frames.

In [11]:
towerToCSV = dict()
root = 'IoBT All Tower Data/'
idx = 0
for tower, data in towerMap.items():
    towerToCSV[tower] = allFiles[idx]
    idx +=1
print(towerToCSV)
intervals = {
    "15min": "FifteenMinDateTime",
    "45min": "FortyFiveMinDateTime",
    "5min": "FiveMinDateTime",
    "H": "HourlyDateTime",
    "30min": "HalfHourlyDateTime"
}

# Create a dictionary to hold the list for each interval.
tower_lists = {}

for interval, datetime_col_name in intervals.items():
    # Create an empty list for the current interval.
    tower_list = []

    for tower, data in towerMap.items():
        # Process data using the provided function.
        print(f"Processing {tower} with {interval} and datetime {datetime_col_name}")
        tower_data = group_data_by_interval(pd.read_csv(root + towerToCSV[tower]), round_interval=interval, datetime_col_name=datetime_col_name)
        tower_list.append(tower_data)

    # Save the resulting list in the dictionary.
    tower_lists[interval] = tower_list
# print(tower_lists['15min'][0])

{'tower13': 'tower13_data_processed_1to9days.csv', 'tower4': 'tower4_data_processed_1to9days.csv', 'tower18': 'tower18_data_processed_1to9days.csv', 'tower24': 'tower24_data_processed_1to9days.csv', 'tower31': 'tower31_data_processed_1to9days.csv', 'tower2': 'tower2_data_processed_1to9days.csv', 'tower12': 'tower12_data_processed_1to9days.csv', 'tower25': 'tower25_data_processed_1to9days.csv', 'tower14': 'tower14_data_processed_1to9days.csv', 'tower10': 'tower10_data_processed_1to9days.csv', 'tower30': 'tower30_data_processed_1to9days.csv', 'tower6': 'tower6_data_processed_1to9days.csv', 'tower8': 'tower8_data_processed_1to9days.csv', 'tower32': 'tower32_data_processed_1to9days.csv', 'tower5': 'tower5_data_processed_1to9days.csv', 'tower28': 'tower28_data_processed_1to9days.csv', 'tower7': 'tower7_data_processed_1to9days.csv', 'tower9': 'tower9_data_processed_1to9days.csv', 'tower26': 'tower26_data_processed_1to9days.csv', 'tower15': 'tower15_data_processed_1to9days.csv', 'tower11': 't

In [ ]:
# find the shortest length
min_len = min(arr.shape[0] for arr in tower_lists['5min'])

# truncate every array/DataFrame to that length
tower_lists['5min'] = [
    arr.iloc[:min_len] if hasattr(arr, 'iloc') else arr[:min_len]
    for arr in tower_lists['5min']
]

# check
print([arr.shape for arr in tower_lists['5min']])
# now they should all be (2604, 4)


[(2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5), (2604, 5)]


In [12]:
print(towerToCSV.values())
tower_labels = [
    f"MAC{num:06d}"
    for num in [int(key[5:]) for key in towerMap.keys()]
]
print(tower_labels)

def csv(k):
  print(f"Nodes: {k}")
  for time, tower_list in tower_lists.items():
    print(f"Time: {time} Processing {intervals[time]}")
    df = pd.DataFrame()
    df['ds'] = tower_list[0][intervals[time]]
    row,col = tower_list[0].shape
    for tower, label in zip(tower_list[:k], tower_labels[:k]):
        if tower.shape[0] != row or tower.shape[1] != col:
          print(tower.shape)
        df[label] = tower['Temperature_2m']
    df.set_index('ds', inplace=True)
    if time == "H":
      df.to_csv(f'temperature_data_60min_{k}.csv')
    else:
      df.to_csv(f'temperature_data_{time}_{k}.csv')
csv(8)
csv(16)
csv(25)

dict_values(['tower13_data_processed_1to9days.csv', 'tower4_data_processed_1to9days.csv', 'tower18_data_processed_1to9days.csv', 'tower24_data_processed_1to9days.csv', 'tower31_data_processed_1to9days.csv', 'tower2_data_processed_1to9days.csv', 'tower12_data_processed_1to9days.csv', 'tower25_data_processed_1to9days.csv', 'tower14_data_processed_1to9days.csv', 'tower10_data_processed_1to9days.csv', 'tower30_data_processed_1to9days.csv', 'tower6_data_processed_1to9days.csv', 'tower8_data_processed_1to9days.csv', 'tower32_data_processed_1to9days.csv', 'tower5_data_processed_1to9days.csv', 'tower28_data_processed_1to9days.csv', 'tower7_data_processed_1to9days.csv', 'tower9_data_processed_1to9days.csv', 'tower26_data_processed_1to9days.csv', 'tower15_data_processed_1to9days.csv', 'tower11_data_processed_1to9days.csv', 'tower21_data_processed_1to9days.csv', 'tower3_data_processed_1to9days.csv', 'tower23_data_processed_1to9days.csv', 'tower27_data_processed_1to9days.csv'])
['MAC000013', 'MAC0

In [13]:
!unzip "Benchmark_STGNN_for_STLF.zip"

Archive:  Benchmark_STGNN_for_STLF.zip
   creating: Benchmark_STGNN_for_STLF/
   creating: Benchmark_STGNN_for_STLF/.git/
  inflating: Benchmark_STGNN_for_STLF/.git/config  
  inflating: Benchmark_STGNN_for_STLF/.git/description  
  inflating: Benchmark_STGNN_for_STLF/.git/HEAD  
   creating: Benchmark_STGNN_for_STLF/.git/hooks/
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/applypatch-msg.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/commit-msg.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/fsmonitor-watchman.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/post-update.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/pre-applypatch.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/pre-commit.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/pre-merge-commit.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/pre-push.sample  
  inflating: Benchmark_STGNN_for_STLF/.git/hooks/pre-rebase.sample  
  inflating: Benchmark_STGN

In [ ]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00


#Install dependencies for Repo

In [14]:

# 1. Full clean uninstall
!pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-geometric pytorch-lightning lightning torch-spatiotemporal pandas

# 2. Install PyTorch 2.0.0 with CUDA 11.7 (matches Colab's default environment)
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1+cu117 --index-url https://download.pytorch.org/whl/cu117

# 3. Install PyG dependencies FOR TORCH 2.0.0
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cu117.html

# 4. Install remaining packages WITH VERSION PINS
!pip install \
  pytorch-lightning==2.0.9 \
  torch-spatiotemporal==0.9.5 \
  pandas==2.2.2 \
  numpy==1.26.4 \
  scipy==1.11.4 \
  dtw-python==1.5.3 \
  einops==0.8.0 \
  pydantic==2.9.2 \
  websockets==14.0.0 \
  fsspec==2024.12.0 \
  gcsfs==2024.12.0 \
  matplotlib==3.9.2 \
  pyinform==0.2.0  \
  optuna==3.1.0

# 5. Create directories
!mkdir -p Benchmark_STGNN_for_STLF/data \
         Benchmark_STGNN_for_STLF/visualization/adj_heatmap \
         Benchmark_STGNN_for_STLF/visualization/forecast

Found existing installation: torch 2.4.1
Uninstalling torch-2.4.1:
  Successfully uninstalled torch-2.4.1
Found existing installation: pandas 1.5.3
Uninstalling pandas-1.5.3:
  Successfully uninstalled pandas-1.5.3
Looking in indexes: https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=e95b99f39932f6a29444e203b0c78650be2da3c1ad6fdc78318a81a0bbe76c6b
  Stored in directory: /root/.cache/pip/wheels/

In [ ]:
!mkdir -p Benchmark_STGNN_for_STLF/data /content/Benchmark_STGNN_for_STLF/visualization/adj_heatmap /content/Benchmark_STGNN_for_STLF/visualization/forecast

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Python 3.11.12
7.34.0


## Run the experiments for GCN Models

In [ ]:
!rm -rf save_inference_result
!mkdir -p save_inference_result
nodes = [8,16,25]
metrics = ["pearson", "euclidean"]
models = {"grugcn_model":"GRUGCN", "tgcn_model": "TGCN", }
redundancies = [0,20,40,60,80,100]
samplings = [5,15,30,45,60]
total = len(models) * len(nodes) * len(redundancies) * len(samplings) * len(metrics)
count = 0
for model,name in models.items():
  for node in nodes:
    for metric in metrics:
      for redundancy in redundancies:
        for sampling in samplings:
          !rm -f /content/Benchmark_STGNN_for_STLF/graph_generation/cache/{metric}_adjacency_matrix.npy
          !time python3 Benchmark_STGNN_for_STLF/SpatioTemporal_TS_with_Graph.py \
          {model} {name}_{redundancy}%_{sampling}mins_{metric}_{node}Nodes --method {metric} --window 8 --hidden_dimension 64 \
          --learning_rate 0.001 --batch_size 64 --horizon 4 --sampling {sampling} --redundancy {redundancy} --nodes {node}
          !python Benchmark_STGNN_for_STLF/tools/evaluate_test_results.py save_inference_result/{name}_{redundancy}%_{sampling}mins_{metric}_{node}Nodes/{metric} > /dev/null 2>&1
          count += 1
          print(f"DONE: Iteration: {count}/{total}\t {name}\t Metric: {metric}\tRedundancy: {redundancy}\tSampling: {sampling}\tNodes: {node}")
!zip -r save_inference_result_GCN_Temperature_Models.zip save_inference_result
from google.colab import files
files.download('save_inference_result_GCN_Temperature_Models.zip')

2.0.0+cu117
Window size is 96 and horizon is 48 for sampling rate of 5 mins
tsl version  : 0.9.5
torch version: 2.0.0+cu117
The chosen model is grugcn_model
Shape of data: (2605, 8)
8
Before downsampling: Shape of df matrix: (2605, 8)
After downsampling: (2605, 8)
args.redundancy: 0
Requested: 0% Redundancy | Actual: 0.0% | Threshold: 2.000
Calculated and saved adjacency matrix to /content/Benchmark_STGNN_for_STLF/graph_generation/cache/pearson_adjacency_matrix.npy
Count of non-zero elements in adjacency matrix: 0
Shape of adj matrix is (8, 8)
Print heatmap of adjacency matrix
Figure(1000x800)
Length of train, val and test splits are: 21, 9, 9
Metadata of pearson is: {'n_channels': 1, 'horizon': 48, 'n_nodes': 8, 'window': 96}

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
You are using a CUDA device ('NVIDIA A1

## Run the experiments for Pure Temporal Models

In [ ]:
!rm -rf save_inference_result
import subprocess
import threading
import time
from google.colab import files
# 2) Define your experiments
nodes = [8, 16, 25]
models = {"var_model": "VAR", "rnn_model": "GRU", "tf_model": "Transformer"}
samplings = [5,15,30,45,60]
count = 0
total = len(nodes) * len(models) * len(samplings)
# 3) Run experiments and async download snapshots
for model, name in models.items():
    for node in nodes:
        for sampling in samplings:
          !time python3 Benchmark_STGNN_for_STLF/SpatioTemporal_TS_with_Graph.py \
          {model} {name}_{sampling}mins_{node}Nodes \
          --window 8 \
          --hidden_dimension 64 \
          --learning_rate 0.001 \
          --batch_size 64 \
          --horizon 4 \
          --sampling {sampling} \
          --nodes {node}
          !python Benchmark_STGNN_for_STLF/tools/evaluate_test_results.py \
          save_inference_result/{name}_{sampling}mins_{node}Nodes
          count += 1
          print(f"DONE: Iteration: {count}/{total} \t Model: {name}\tSampling: {sampling}\tNodes: {node}")
!zip -r save_inference_result_Temperature_Temporal_Models.zip save_inference_result
from google.colab import files
files.download('save_inference_result_Temperature_Temporal_Models.zip')

2.0.0+cu117
Window size is 96 and horizon is 48 for sampling rate of 5 mins
tsl version  : 0.9.5
torch version: 2.0.0+cu117
The chosen model is var_model
Shape of data: (2605, 8)
8
Length of train, val and test splits are: 21, 9, 9
Metadata of None is: {'n_channels': 1, 'horizon': 48, 'n_nodes': 8, 'window': 96}

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-05-15 05:00:11.288135: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You